## Using Python to perform regressions on all possible independent variable subsets

Kenneth Burchfiel

First uploaded to GitHub on 2021-03-19

MIT license (gov_data_compilation_3.csv dataset is in the public domain)

This program analyzes what combination of 7 independent variables may best explain differences among U.S. states in domestic (i.e. state-to-state) migration rates in 2018. The 7 variables, obtained from federal government data and thus believed to be in the public domain, are as follows:

1. 'rpp_2018' (regional price parities, which I understand to be a measure of the relative cost of of living in different states)

2. 'unemployment_2018' (the 2018 unemployment rate)

3. '2016_trump_vote', (what percent of votes in the 2016 presidential election in that state were for Donald Trump)

4. 'gdp_pct_change_2017_2018' (the percentage change in that state's gross domestic product from 2017 to 2018)

5. 'personal_income_per_capita_pct_change_2017_2018',

6. 'married_couple_families_as_share_of_households_2018',

7. 'fertility_rate_per_1000_2018'


Although the findings from this regression are interesting, the main purpose of this program is to demonstrate how Python can make it both easier and faster to perform lots of regressions on a set of variables. The program accomplishes the following steps:

Step 1: Read a CSV file (containing 7 independent variables) into a Pandas DataFrame. 

Step 2: Find all possible combinations of those variables that can be entered into a linear regression

Step 3: Perform a linear regression on each of those variables, and append the regression output to a new list

Step 4: Convert that list into a DataFrame, allowing the different models to be compared

## Step 1

In [1]:
import pandas as pd
import statsmodels.api as sm
df_states = pd.read_csv('gov_data_compilation_3.csv',index_col='State') # gov_data_compilation_3.csv contains various statistics accessed from federal government sources (the Bureau of Economic Analysis; the Federal Election Commission; the US Census Bureau; and the Bureau of Labor Statistics). Because they were created by the U.S. government, I believe they are all in the public domain.
df_states 

,rpp_2018,unemployment_2018,2016_trump_vote,gdp_pct_change_2017_2018,personal_income_per_capita_pct_change_2017_2018,married_couple_families_as_share_of_households_2018,fertility_rate_per_1000_2018,domestic_migration_rate_2018,pop_pct_chg_2017_2018
State,,,,,,,,,
Alabama,85.9,3.9,0.620831,2.251346,3.848464,0.474,58,1.081522,0.270695
Alaska,105.7,5.9,0.512815,0.194602,5.231292,0.502,62,-14.587355,-0.616601
Arizona,96.4,4.8,0.486716,3.938298,4.297047,0.477,57,11.845206,1.618624
Arkansas,84.7,3.7,0.605719,1.520739,4.317724,0.479,57,0.763257,0.279475
California,116.2,4.3,0.316171,3.068594,4.862066,0.494,47,-3.940138,0.261928
Colorado,101.3,3.0,0.432510,4.369849,5.217697,0.491,52,7.526914,1.414890
Connecticut,105.5,3.9,0.409269,0.389476,4.068240,0.472,44,-6.711439,-0.049730
Delaware,98.7,3.7,0.417127,2.120415,3.727372,0.473,47,7.127912,0.904661
District of Columbia,115.2,5.7,0.040875,2.038917,2.310227,0.254,41,-0.455440,0.955669


In [2]:
df_states.dtypes # Confirms that all the variables are numerical in nature, which is important for the regression analysis

rpp_2018                                               float64
unemployment_2018                                      float64
2016_trump_vote                                        float64
gdp_pct_change_2017_2018                               float64
personal_income_per_capita_pct_change_2017_2018        float64
married_couple_families_as_share_of_households_2018    float64
fertility_rate_per_1000_2018                             int64
domestic_migration_rate_2018                           float64
pop_pct_chg_2017_2018                                  float64
dtype: object

## Step 2

I will next create a function that, given 7 independent variables, produces all possible subsets of those variables, then adds those subsets (except for the empty subset) to a list. Since each variable can either be included or not included (2 possible conditions), the total number of non-empty subsets = 2^7 -1 = 127.

The function uses a series of for loops to store 2 different values (0 and 1) in 7 different variables (include_var_1, include_var_2 . . . include_var_7). Within those sets, if a variable equals 1, its corresponding DataFrame variable will be included in a list (var_list). Each var_list with at least one item will then be added to another list (subset_list).

To better visualize the function's output, here are the different lists of include_var values that would be created if there were only 3 variables (A, B, and C). I also included the subset of variables that would correspond to each list.

    include_var_0 = 1

        include_var_1 = 1

            include_var_2 = 1
            
                [Output: 1, 1, 1]

                [Corresopnding variable list: A, B, C]

            include_var_2 = 0

                [Output: 1, 1, 0]

                [Corresopnding variable list: A, B]

        include_var_1 = 0

            include_var_2 = 1

                [Output: 1, 0, 1]

                [Corresopnding variable list: A, C]

            include_var_2 = 0

                [Output: 1, 0, 0]

                [Corresopnding variable list: A]

    include_var_0 = 0

        include_var_1 = 1

                include_var_2 = 1

                    [Output: 0, 1, 1]

                    [Corresopnding variable list: B, C]

                include_var_2 = 0

                    [Output: 0, 1, 0]

                    [Corresopnding variable list: B]

            include_var_1 = 0

                include_var_2 = 1

                    [Output: 0, 0, 1]

                    [Corresopnding variable list: C]

                include_var_2 = 0

                    [Output: 0, 0, 0]
                    
                    [Corresopnding variable list: [] (not included)]




A disadvantage to this function is that it is hard-coded for 7 variables. I imagine that there is a way to create a recursive function that allows for all subsets of an arbitrarily long set of variables to be created. However, I will use the hard-coded strategy for now. 

In [3]:
def create_7_var_list_subsets(variable_list): 
    subset_list = []
    for include_var_0 in range (2): # Creates two conditions: include_var_0 = 1, and include_var_0 = 0
        for include_var_1 in range (2):
            for include_var_2 in range (2):
                for include_var_3 in range (2):
                    for include_var_4 in range (2):
                        for include_var_5 in range (2):
                            for include_var_6 in range (2):
                                var_list = [] # This list will store the variable subset for this particular result of the 7 for loops.
                                if include_var_0 == 1:
                                    var_list.append(variable_list[0])
                                if include_var_1 == 1:
                                    var_list.append(variable_list[1])
                                if include_var_2 == 1:
                                    var_list.append(variable_list[2])
                                if include_var_3 == 1:
                                    var_list.append(variable_list[3])
                                if include_var_4 == 1:
                                    var_list.append(variable_list[4])
                                if include_var_5 == 1:
                                    var_list.append(variable_list[5])
                                if include_var_6 == 1:
                                    var_list.append(variable_list[6])                                  
                                # print(var_list) for debugging
                                if len(var_list) > 0: # Excludes the empty subset
                                # from the subset list
                                    subset_list.append(var_list) # Adds this particular subset to the overall list of subsets
    return subset_list           


 

Next, I will use a list of my DataFrame's columns to create a list of 7 independent variables that can be entered into the create_7_var_list_subsets function.

In [4]:
ind_variables_list=[]
for i in range(7):
    ind_variables_list.append(df_states.columns[i])
ind_variables_list # Prints out the list of 7 independent variables

['rpp_2018',
 'unemployment_2018',
 '2016_trump_vote',
 'gdp_pct_change_2017_2018',
 'personal_income_per_capita_pct_change_2017_2018',
 'married_couple_families_as_share_of_households_2018',
 'fertility_rate_per_1000_2018']

I will now create all possible subsets of these 7 independent variables using the create_7_var_list_subsets function.

In [5]:
ind_regression_models = create_7_var_list_subsets(ind_variables_list)
for model in ind_regression_models:
    print(model)                 
print(len(ind_regression_models)) # Verifies that 127 (2^7 - 1) models have been created

['fertility_rate_per_1000_2018']
['married_couple_families_as_share_of_households_2018']
['married_couple_families_as_share_of_households_2018', 'fertility_rate_per_1000_2018']
['personal_income_per_capita_pct_change_2017_2018']
['personal_income_per_capita_pct_change_2017_2018', 'fertility_rate_per_1000_2018']
['personal_income_per_capita_pct_change_2017_2018', 'married_couple_families_as_share_of_households_2018']
['personal_income_per_capita_pct_change_2017_2018', 'married_couple_families_as_share_of_households_2018', 'fertility_rate_per_1000_2018']
['gdp_pct_change_2017_2018']
['gdp_pct_change_2017_2018', 'fertility_rate_per_1000_2018']
['gdp_pct_change_2017_2018', 'married_couple_families_as_share_of_households_2018']
['gdp_pct_change_2017_2018', 'married_couple_families_as_share_of_households_2018', 'fertility_rate_per_1000_2018']
['gdp_pct_change_2017_2018', 'personal_income_per_capita_pct_change_2017_2018']
['gdp_pct_change_2017_2018', 'personal_income_per_capita_pct_change_201

## Step 3

The following function takes a list of independent variable subsets; performs a linear regression on each one; stores various elements of that regression output into a dictionary; and appends that dictionary into a list of dictionaries.

In [6]:
def subset_regressions(subset_list, dv, df):
    regression_dict_list = []
    for i in range(len(subset_list)):
            y = dv
            x = df[subset_list[i]] # This method of adding a list of IVs is based on a Stack Overflow answer by unutbu: https://stackoverflow.com/a/29186780/13097194
            x = sm.add_constant(x)
            output = sm.OLS(y,x)
            results = output.fit()
            regression_dict = {}
            regression_dict['IVs']=subset_list[i] # Stores the list of independent variables used in the regression into the dictionary
            regression_dict['rsquared']=results.rsquared
            regression_dict['adj_rsquared']=results.rsquared_adj
            regression_dict['summary']=results.summary()
            regression_dict_list.append(regression_dict)
    return regression_dict_list


## Step 4

I will now enter the list of independent variable subsets created earlier into the subset_regression function.

In [7]:
regression_dv = df_states['domestic_migration_rate_2018'] # Using the 2018 domestic migration rate as the dependent variable
regression_table = subset_regressions(ind_regression_models,regression_dv,df_states)
df_regressions = pd.DataFrame(regression_table) # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html
pd.set_option('display.max_rows',150)
df_regressions.head(10) # Listed in the order the regressions were generated

,IVs,rsquared,adj_rsquared,summary
0,[fertility_rate_per_1000_2018],0.002458,-0.017900,OLS Regressio...
1,[married_couple_families_as_share_of_household...,0.007845,-0.012403,OLS Regressio...
2,[married_couple_families_as_share_of_household...,0.019533,-0.021320,OLS Regressio...
3,[personal_income_per_capita_pct_change_2017_2018],0.000004,-0.020404,OLS Regressio...
4,[personal_income_per_capita_pct_change_2017_20...,0.003735,-0.037776,OLS Regressio...
5,[personal_income_per_capita_pct_change_2017_20...,0.008665,-0.032640,OLS Regressio...
6,[personal_income_per_capita_pct_change_2017_20...,0.020229,-0.042309,OLS Regressio...
7,[gdp_pct_change_2017_2018],0.379965,0.367311,OLS Regressio...
8,"[gdp_pct_change_2017_2018, fertility_rate_per_...",0.404940,0.380145,OLS Regressio...
9,"[gdp_pct_change_2017_2018, married_couple_fami...",0.385675,0.360078,OLS Regressio...


I will now sort this table of regressions in descending order by their adjusted r-squared in order to see which subsets may best explain the variation in the 2018 U.S. domestic migration rate. 

*Note: I say 'may explain' because there is always a risk of conflating correlation with causation. It is possible that the independent variables can help explain the domestic migration rate; however, it is also possible that the domestic migraiton rate instead plays a role in determining the independent variables, or that some third variable is behind the relationship between each independent variable and the dependent variable.*

In [8]:
df_regressions.sort_values('adj_rsquared', ascending=False, inplace=True)
df_regressions.head()


,IVs,rsquared,adj_rsquared,summary
108,"[rpp_2018, unemployment_2018, gdp_pct_change_2...",0.603244,0.559161,OLS Regressio...
124,"[rpp_2018, unemployment_2018, 2016_trump_vote,...",0.606293,0.552606,OLS Regressio...
110,"[rpp_2018, unemployment_2018, gdp_pct_change_2...",0.603873,0.549855,OLS Regressio...
126,"[rpp_2018, unemployment_2018, 2016_trump_vote,...",0.606295,0.542203,OLS Regressio...
76,"[rpp_2018, gdp_pct_change_2017_2018, personal_...",0.572915,0.535778,OLS Regressio...


Now that the table is sorted, the best-performing model, which happens to use 5 variables (rpp_2018; unemployment_2018; gdp_pct_change_2017_2018; personal_income_per_capita_pct_change_2017_2018; and fertility_rate_per_1000_2018) is at the top. The following line accesses the statsmodels summary output for that model.

In [9]:
df_regressions.iloc[0,3]

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                 
========================================================================================
Dep. Variable:     domestic_migration_rate_2018   R-squared:                       0.603
Model:                                      OLS   Adj. R-squared:                  0.559
Method:                           Least Squares   F-statistic:                     13.68
Date:                          Fri, 19 Mar 2021   Prob (F-statistic):           3.94e-08
Time:                                  16:57:59   Log-Likelihood:                -140.64
No. Observations:                            51   AIC:                             293.3
Df Residuals:                                45   BIC:                             304.9
Df Model:                                     5                                         
Covariance Type:                      nonrobust                                         
===================================================================================================================
                                                      coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------------------
const                                              45.4524     11.623      3.911      0.000      22.042      68.863
rpp_2018                                           -0.2921      0.072     -4.073      0.000      -0.436      -0.148
unemployment_2018                                  -1.3526      0.729     -1.855      0.070      -2.821       0.116
gdp_pct_change_2017_2018                            3.2564      0.455      7.156      0.000       2.340       4.173
personal_income_per_capita_pct_change_2017_2018    -1.5053      0.615     -2.446      0.018      -2.745      -0.266
fertility_rate_per_1000_2018                       -0.2565      0.121     -2.112      0.040      -0.501      -0.012
==============================================================================
Omnibus:                        5.704   Durbin-Watson:                   1.701
Prob(Omnibus):                  0.058   Jarque-Bera (JB):                5.092
Skew:                           0.493   Prob(JB):                       0.0784
Kurtosis:                       4.193   Cond. No.                     2.27e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.27e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

Finally, I will save this sorted table as a CSV file.

In [10]:
df_regressions.to_csv('domestic_migration_regressions.csv')

The regression results should be interpreted with caution for a number of reasons:

1. It is possible that the 5-variable model featured above performed the best out of all 127 due to random chance, and not because it is a valid predictor of variation in state migration rates. There is also a risk that the model has overfit to this particular dataset.

In order to guard against this possibility, the program could be revised so that each model is trained on one portion of the data, then tested on another portion (i.e. using k-fold cross-validation).

2. This model may be less accurate for subsequent years (2019, 2020, etc.).

3. Correlation is no guarantee of causation, as discussed above.


However, as noted earlier, the main purpose of this program is to demonstrate how Python makes it easy to perform hundreds of regressions in just seconds. It should not prove difficult to repurpose this program for other analyses.